In [1]:
!pip install cassandra-driver

In [1]:
from json import loads
import cassandra
cassandra. __version__

'3.25.0'

In [3]:
from cassandra.cluster import Cluster

cluster = Cluster()
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS mydb WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

In [20]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
from elasticsearch import Elasticsearch
import time
import re
import json
from json import loads

consumer = KafkaConsumer(
    'persistence',
     bootstrap_servers=['192.168.1.3:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group3',
     api_version=(0,10),
     value_deserializer=lambda x: loads(x.decode('utf-8')))

# Make a Dataframe to store the consumer contents:

In [69]:
import pandas as pd
elasticSearch = Elasticsearch([{'host':'localhost','port':9200}])
df=pd.DataFrame(columns=['hashtag','id','year', 'month','day','hour','minute'])
df

,hashtag,id,year,month,day,hour,minute


In [14]:
for message in consumer:
    message = message.value
    print(message)

KeyboardInterrupt: 

In [150]:
producer = KafkaProducer(bootstrap_servers=['192.168.1.3:9092'],key_serializer=str.encode,api_version=(0,10),
                         value_serializer=lambda x:dumps(x).encode('utf-8'))

In [70]:
import datetime

for message in consumer:
    message = message.value
    if 'content' not in message:
        continue
    
    #Inserting data to Elastic Search:
    try:
        elasticSearch.index(index='twitter_bors',doc_type='twitt',body = message, id =message['id'])
    except:
        print("An exception occurred")
    
    
    #At the same time, inserting data to data frame to be injected to Cassandra tables:
    id=int(message['id'])
    content=message['content']
    date = datetime.datetime.strptime(message['sendTimePersian'], "%Y/%m/%d  %H:%M")
    year=date.year
    month=date.month
    day=date.day
    hour=date.hour
    minute=date.minute
    
    for h in message['hashtags']:
        new_row={'hashtag':h,'id':id,'year':year,'month':month,'day':day,'hour':hour,'minute':minute,}
        df=df.append(new_row, ignore_index=True)
        
    producer.send('statistics', key=message['id'],value=message)


c:\users\alija\miniconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


KeyboardInterrupt: 

In [71]:
df

,hashtag,id,year,month,day,hour,minute
0,شلرد,321482732,1400,4,19,16,5
1,فاهواز,321482718,1400,4,19,16,5
2,شکف,321482707,1400,4,19,16,5
3,سیدکو,321482677,1400,4,19,16,4
4,آینده,321482618,1400,4,19,16,4
5,حریل,321482596,1400,4,19,16,4
6,بزاگرس,321482581,1400,4,19,16,4
7,شلرد,321283095,1400,4,19,6,49
8,برکت,321482556,1400,4,19,16,4
9,فنوال,321482552,1400,4,19,16,4


In [8]:
df[df.iloc[:,0]=='شپلی']

,hashtag,id,year,month,day,hour,minute
402,شپلی,320210026,1400,4,17,0,3


# Elastic Search Sample Queries

In [37]:
res = elasticSearch.get(index="twitter_bors", id='320813014')
print(res['_source'])

{'id': '320813014', 'sendTime': '2021-07-09T14:24:15Z', 'sendTimePersian': '1400/04/18 18:54', 'senderName': 'داود زنجانی', 'senderUsername': 'al33541575masi', 'senderProfileImage': '7e0981af-5444-4e56-bba6-d75dcbd8725d', 'content': '#کاما :دوستان سهامدار کاما اگر یکی از دو خبر افزایش سرمایه 84درصدی مرحله اول یا بورسی شدن سهام کانی کربن طبس مورد تایید سازمان بورس قرار گیرد در کوتاه مدت حداقل قیمت کاما کمتر از 2000تومان نخواهد بود.مراقب سهام ارزشمند خود باشید واگر خرید یا اضافه نمی کنید بهیچ عنوان فروشنده نباشید.', 'type': 'twit', 'scoredPostDate': '1625840725903', 'finalPullDatePersian': '', 'hashtags': ['کاما'], 'links': [], 'keywords': ['سازمان', 'دوستان', 'درصدی', 'خرید', 'خبر', 'تومان', 'تایید', 'بورسی', 'افزایش', 'اضافه', 'ارزشمند', ':', '۸۴', 'بورس']}


In [51]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': [320813026,320812966]}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 2 Hits:
{'id': '320813026', 'sendTime': '2021-07-09T14:24:33Z', 'sendTimePersian': '1400/04/18 18:54', 'senderName': 'تهران بورس', 'senderUsername': 'nabatali', 'senderProfileImage': 'fb8f1f9d-a66d-480b-88b2-e0c150a236ea', 'content': '#شپلی دوستان عزیز شرمنده یک سوالی دارم ممنون میشم جوابم رو بدید..روزانه چقدر میشه از حساب به حساب کارگزاری انتقال داد!؟؟۵۰میلیون؟؟؟چجوری میشه پول بیشتری در یک روز انتقال داد؟', 'type': 'twit', 'scoredPostDate': '1625840659777', 'finalPullDatePersian': '', 'hashtags': ['شپلی'], 'links': [], 'keywords': ['روزانه', 'سوالی', 'شپلی', 'روز', 'دوستان', 'جوابم', '؟؟؟', '؟', '۵۰']}
{'id': '320812966', 'sendTime': '2021-07-09T14:22:17Z', 'sendTimePersian': '1400/04/18 18:52', 'senderName': 'مهدی ایمنی', 'senderUsername': 'mahdiimeni', 'senderProfileImage': '3194ea7e-4fef-4f8a-bdc7-46131537f17b', 'content': '#وخارزم \n- از آخرین باری که تحلیل کردم در کمتر از یک ماه 50 درصد رشد کرده!', 'type': 'twit', 'imageUid': 'fbf2fb34-6810-435d-acab-99769e93929b', 'mediaCont

# تمام پست های مربوط به یک هشتگ

In [56]:
query="CREATE TABLE IF NOT EXISTS mydb.hashtag(\
hashtag text,\
id int,\
PRIMARY KEY (hashtag, id)\
);"

session.execute(query)

In [57]:
session.execute("TRUNCATE mydb.hashtag;")

In [58]:
import pandas as pd
import numpy as np
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel

ps = session.prepare("INSERT INTO mydb.hashtag (hashtag,id) VALUES (?, ?)")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for i in range(np.int32(np.floor(df.shape[0]/100))):
    batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
    for j in range(100):
        batch.add(ps,(df.iloc[i*100+j,0], df.iloc[i*100+j,1]))
    session.execute(batch)

batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
for i in range(np.int32(np.floor(df.shape[0]/100))*100,df.shape[0]):
    batch.add(ps,(df.iloc[i,0], df.iloc[i,1]))
    
session.execute(batch)

In [61]:
result=session.execute("select * from mydb.hashtag ") #where hashtag='شپلی'
for row in result:
    print(row)

Row(hashtag='شپلی', id=320813026)
Row(hashtag='غصینو', id=320812949)
Row(hashtag='زقیام', id=320812911)
Row(hashtag='کیمیا', id=320812885)
Row(hashtag='شاخص_بورس', id=318770833)
Row(hashtag='کاما', id=320813014)
Row(hashtag='وخارزم', id=320812946)
Row(hashtag='وخارزم', id=320812966)
Row(hashtag='فملی', id=315327806)


In [62]:
result=session.execute("select * from mydb.hashtag where hashtag='فملی'") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row.id)

315327806


In [64]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 1 Hits:
{'id': '315327806', 'sendTime': '2021-06-30T15:21:14Z', 'sendTimePersian': '1400/04/09 19:51', 'retwitSendTime': '2021-07-09T14:23:31Z', 'retwitSendTimePersian': '1400/04/18 18:53', 'retwitSenderName': 'ramsess', 'retwitSenderUsername': 'ramsess', 'retwitSenderProfileImage': 'default', 'senderName': 'ramsess', 'senderUsername': 'ramsess', 'senderProfileImage': 'default', 'content': '#فملی : *طولانی شدن فرآیند مذاکرات وین به افزایش قیمت دلار منجر شد* \n\n *کاهش خوشبینی دلار به احیای برجام* \n\n۰۸/۰۴/۱۴۰۰\nکسب و کار نیوز- \n *قیمت دلار در معاملات به 25 هزار و 76 تومان افزایش یافت. ورود دلار به کانال 25 هزار تومان نشان می دهد که انتظارات از مذاکرات وین تغییر کرده و به دلیل طولانی شدن فرآیند مذاکرات احتمال نتیجه بخش نبودن آن نیز مطرح است.* \nنویسنده : *شایلی قرائی* \n *به گزارش کسب و کار نیوز ، یک کارشناس اقتصادی در این رابطه می گوید: در شرایط فعلی موضوعی که بیشتر از هر چیزی بر قیمت ارز تاثیر دارد برآوردها از نتیجه مذاکرات وین و احیای برجام است. در حال حاضر نحوه پیشبرد مذاکرات 

# تمام پست های مربوط به یک هشتگ در یک بازه زمانی خاص

In [67]:
query="CREATE TABLE IF NOT EXISTS mydb.hashtag_interval(\
hashtag text,\
day int,\
hour int,\
minute int,\
id int,\
PRIMARY KEY (hashtag, day, hour, minute, id)\
);"

session.execute(query)

In [ ]:
session.execute("TRUNCATE mydb.hashtag_interval;")

In [84]:
import pandas as pd
import numpy as np
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel

ps = session.prepare("INSERT INTO mydb.hashtag_interval (hashtag, day, hour, minute, id) VALUES (?, ?, ?, ?, ?)")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for i in range(np.int32(np.floor(df.shape[0]/100))):
    batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
    for j in range(100):
        batch.add(ps,(df.iloc[i*100+j,0], df.iloc[i*100+j,4], df.iloc[i*100+j,5], df.iloc[i*100+j,6], df.iloc[i*100+j,1]))
    session.execute(batch)

batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
for i in range(np.int32(np.floor(df.shape[0]/100))*100,df.shape[0]):
    batch.add(ps,(df.iloc[i,0], df.iloc[i,4], df.iloc[i,5], df.iloc[i,6], df.iloc[i,1]))
    
session.execute(batch)

In [85]:
result=session.execute("select * from mydb.hashtag_interval ") #where hashtag='شپلی'
for row in result:
    print(row)

Row(hashtag='سیدکو', day=19, hour=16, minute=4, id=321482677)
Row(hashtag='فنوال', day=19, hour=16, minute=4, id=321482552)
Row(hashtag='بزاگرس', day=19, hour=16, minute=4, id=321482581)
Row(hashtag='برکت', day=19, hour=16, minute=4, id=321482556)
Row(hashtag='شکف', day=19, hour=16, minute=5, id=321482707)
Row(hashtag='وپسا', day=19, hour=16, minute=5, id=321482807)
Row(hashtag='فاهواز', day=19, hour=16, minute=5, id=321482718)
Row(hashtag='شلرد', day=19, hour=6, minute=49, id=321283095)
Row(hashtag='شلرد', day=19, hour=16, minute=5, id=321482732)
Row(hashtag='آینده', day=19, hour=16, minute=4, id=321482618)
Row(hashtag='حریل', day=19, hour=16, minute=4, id=321482596)


In [97]:
result=session.execute("select * from mydb.hashtag_interval where hashtag='شلرد' and day=19 and hour>4 and hour <18") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row)

Row(hashtag='شلرد', day=19, hour=6, minute=49, id=321283095)
Row(hashtag='شلرد', day=19, hour=16, minute=5, id=321482732)


In [98]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 2 Hits:
{'id': '321482732', 'sendTime': '2021-07-10T11:35:22Z', 'sendTimePersian': '1400/04/19 16:05', 'parentSendTime': '2021-07-10T11:33:36Z', 'parentSendTimePersian': '1400/04/19 16:03', 'parentId': '321482467', 'parentSenderName': 'بورسی', 'parentSenderUsername': 'vahidkhobpanah', 'parentSenderProfileImage': 'default', 'parentContent': 'بی سواد کجا پایینتر برو درست و حسابی گزارشو بخون تنی یک میلیون هم گرونتر', 'senderName': 'Unknown', 'senderUsername': 'unknown12345', 'senderProfileImage': 'default', 'content': '#شلرد یعنی تو باسوادتری🤣🤣🤣', 'type': 'quote', 'finalPullDatePersian': '', 'hashtags': ['شلرد'], 'links': [], 'keywords': ['']}
{'id': '321283095', 'sendTime': '2021-07-10T02:19:30Z', 'sendTimePersian': '1400/04/19 06:49', 'retwitSendTime': '2021-07-10T11:34:13Z', 'retwitSendTimePersian': '1400/04/19 16:04', 'retwitSenderName': 'بورسی', 'retwitSenderUsername': 'vahidkhobpanah', 'retwitSenderProfileImage': 'default', 'senderName': 'بورسی', 'senderUsername': 'vahidkhobpana

# تمام پست ها در یک بازه زمانی خاص

In [136]:
query="CREATE TABLE IF NOT EXISTS mydb.total_interval(\
year int,\
month int,\
day int,\
hour int,\
minute int,\
id int,\
PRIMARY KEY ((year, month), day, hour, minute, id)\
);"

session.execute(query)

In [135]:
# session.execute("TRUNCATE mydb.total_interval;")
session.execute("drop table mydb.total_interval;")

In [104]:
df.head()

,hashtag,id,year,month,day,hour,minute
0,شلرد,321482732,1400,4,19,16,5
1,فاهواز,321482718,1400,4,19,16,5
2,شکف,321482707,1400,4,19,16,5
3,سیدکو,321482677,1400,4,19,16,4
4,آینده,321482618,1400,4,19,16,4


In [137]:
import pandas as pd
import numpy as np
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel

ps = session.prepare("INSERT INTO mydb.total_interval (year, month, day, hour, minute, id) VALUES (?, ?, ?, ?, ?, ?)")
batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)

for i in range(np.int32(np.floor(df.shape[0]/100))):
    batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
    for j in range(100):
        batch.add(ps,(df.iloc[i*100+j,2], df.iloc[i*100+j,3], df.iloc[i*100+j,4], df.iloc[i*100+j,5], df.iloc[i*100+j,6], df.iloc[i*100+j,1]))
    session.execute(batch)

batch = BatchStatement(consistency_level=ConsistencyLevel.QUORUM)
for i in range(np.int32(np.floor(df.shape[0]/100))*100,df.shape[0]):
    batch.add(ps,(df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5], df.iloc[i,6], df.iloc[i,1]))
    
session.execute(batch)

In [138]:
result=session.execute("select * from mydb.total_interval ") #where hashtag='شپلی'
for row in result:
    print(row)

Row(year=1400, month=4, day=19, hour=6, minute=49, id=321283095)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482552)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482556)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482581)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482596)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482618)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482677)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482707)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482718)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482732)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482807)


In [142]:
result=session.execute("select * from mydb.total_interval where year=1400 and month=4 ") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row)

Row(year=1400, month=4, day=19, hour=6, minute=49, id=321283095)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482552)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482556)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482581)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482596)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482618)
Row(year=1400, month=4, day=19, hour=16, minute=4, id=321482677)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482707)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482718)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482732)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482807)


In [143]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

Got 11 Hits:
{'id': '321482552', 'sendTime': '2021-07-10T11:34:09Z', 'sendTimePersian': '1400/04/19 16:04', 'senderName': 'A.NOURI', 'senderUsername': 'nouriiraj', 'senderProfileImage': '01307f71-5fea-4be1-8b42-b25d8056e48c', 'content': '#فنوال اقای بازیگر حقوقی خودشرکت \nبچرخ تا بچرخیم تو .... پول شرکت وملت دستت است و باید فردای مجمع به خاطر این حماقت و تعلل در رشدسهم باید به خیلی از نباید های سهامداران تو مجمع جوابگو باشی، ولی ما برایمان فرقی نمیکند این مجمع هیچ،مجمع بعدی هم با تو هستیم \nولی یه چیز بگم ا ح م ق تر از حقوقی فنوال در هیچ سهام در طول 15 سال سابقه بورسی خود ندیده بودم \nبا این لجاجت :\n1- بازه زمانی  اصلاح قیمتی  فنوال  تموم شده و به زور فنوال را فریزکردید و با این کا و بازی ها و معاملات لگاریتمی و الگوریتی دستتان خوانده شده.\n2- باعث شدید گپ قیمتی فنوال پر نشه و  با این کار افزایش سرمایه از سود و تجدید را به تاخیر میاندازید.\n3-مزایده بار سوم فپارس که از لحاظ قانونی بازه زمانی برگزاری آن سپری شده و به خاطر تصاحب تمام کارخانه توسط تیم محمدیون تعلل میکنن.(سه حالت برای فپا

In [148]:
result=session.execute("select * from mydb.total_interval where year=1400 and month=4 and day=19 and hour=16 and minute>4") #
ids_list=[]
for row in result:
    ids_list.append(row.id)
    print(row)

Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482707)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482718)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482732)
Row(year=1400, month=4, day=19, hour=16, minute=5, id=321482807)


In [ ]:
res = elasticSearch.search(index="twitter_bors", body={"query": {"ids": { 'values': ids_list}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit['_source'])

# Finding Statistics using Cassandra

In [151]:
result=session.execute("select count(*) from mydb.total_interval where year=1400 and month=4 and day=19 and hour=16 and minute>4") #
for row in result:
    print(row)

Row(count=4)


In [153]:
result=session.execute("select avg(hour) from mydb.total_interval where year=1400 and month=4 and day=19") #
for row in result:
    print(row)

Row(system_avg_hour=15)
